In [1]:
import os
print(os.getcwd());os.chdir('../')
print(os.getcwd())

/data01/ch6845/MarcoPolo_revision/experiments
/data01/ch6845/MarcoPolo_revision


In [2]:
import sys

import numpy as np
import pandas as pd

from scipy.io import mmread

import MarcoPolo.QQscore as QQ

In [3]:
dataset_name_all=[
'Kohinbulk_filtered',
'HumanLiver_filtered',
'Zhengmix8eq_filtered'  
]

In [4]:
mode=2

In [5]:
#gene_info=pd.read_csv('datasets/Homo_sapiens.gene_info.gz',sep='\t')

In [6]:
dataset_name_all=dataset_name_all+["TabulaAorta_filtered",
"TabulaBladder_filtered",
"TabulaBrainMyeloid_filtered",
"TabulaBrainNonMyeloid_filtered",
"TabulaDiaphragm_filtered",
"TabulaFat_filtered",
"TabulaHeart_filtered",
"TabulaKidney_filtered",
"TabulaLargeIntestine_filtered",
"TabulaLimbMuscle_filtered",
"TabulaLiver_filtered",
"TabulaLung_filtered",
"TabulaMammaryGland_filtered",
"TabulaMarrow_filtered",
"TabulaPancreas_filtered",
"TabulaSkin_filtered",
"TabulaSpleen_filtered",
"TabulaThymus_filtered",
"TabulaTongue_filtered",
"TabulaTrachea_filtered"]

In [8]:
dataset_simul_all=[]
for ncells_total in [1000,2000,5000,10000]:
    for prop in ['1e-2','5e-3','1e-3','5e-4']:
        for i in range(1,10+1):
            dataset_simul_all.append('Simul_{}_{}_{}_filtered'.format(ncells_total,prop,i))

In [9]:
dataset_name_all=dataset_name_all+dataset_simul_all

In [10]:
dataset_name_all

['Kohinbulk_filtered',
 'HumanLiver_filtered',
 'Zhengmix8eq_filtered',
 'TabulaAorta_filtered',
 'TabulaBladder_filtered',
 'TabulaBrainMyeloid_filtered',
 'TabulaBrainNonMyeloid_filtered',
 'TabulaDiaphragm_filtered',
 'TabulaFat_filtered',
 'TabulaHeart_filtered',
 'TabulaKidney_filtered',
 'TabulaLargeIntestine_filtered',
 'TabulaLimbMuscle_filtered',
 'TabulaLiver_filtered',
 'TabulaLung_filtered',
 'TabulaMammaryGland_filtered',
 'TabulaMarrow_filtered',
 'TabulaPancreas_filtered',
 'TabulaSkin_filtered',
 'TabulaSpleen_filtered',
 'TabulaThymus_filtered',
 'TabulaTongue_filtered',
 'TabulaTrachea_filtered',
 'Simul_1000_1e-2_1_filtered',
 'Simul_1000_1e-2_2_filtered',
 'Simul_1000_1e-2_3_filtered',
 'Simul_1000_1e-2_4_filtered',
 'Simul_1000_1e-2_5_filtered',
 'Simul_1000_1e-2_6_filtered',
 'Simul_1000_1e-2_7_filtered',
 'Simul_1000_1e-2_8_filtered',
 'Simul_1000_1e-2_9_filtered',
 'Simul_1000_1e-2_10_filtered',
 'Simul_1000_5e-3_1_filtered',
 'Simul_1000_5e-3_2_filtered',
 'Sim

In [11]:
for dataset_name in dataset_name_all[:]:
    print(dataset_name)
    
    path='datasets/extract/{}'.format(dataset_name)
    path
    
    exp_data=mmread('{}.data.counts.mm'.format(path)).toarray().astype(float)
    with open('{}.data.col'.format(path),'r') as f: exp_data_col=[i.strip().strip('"') for i in f.read().split()]
    with open('{}.data.row'.format(path),'r') as f: exp_data_row=[i.strip().strip('"') for i in f.read().split()]
    assert exp_data.shape==(len(exp_data_row),len(exp_data_col))
    assert len(set(exp_data_row))==len(exp_data_row)
    assert len(set(exp_data_col))==len(exp_data_col)        

    exp_data_meta=pd.read_csv('{}.metadatacol.tsv'.format(path),sep='\t')

    cell_size_factor=pd.read_csv('{}.size_factor.tsv'.format(path),sep='\t',header=None)[0].values.astype(float)#.reshape(-1,1)

    x_data_intercept=np.array([np.ones(exp_data.shape[1])]).transpose()
    x_data_null=np.concatenate([x_data_intercept],axis=1)

    result_list,gamma_list_list,delta_log_list_list,beta_list_list=QQ.read_QQscore(path,[1,mode])

    gamma_list=gamma_list_list[-1]    

    gamma_argmax_list=QQ.gamma_list_exp_data_to_gamma_argmax_list(gamma_list,exp_data)#gamma_argmax_list=QQ.gamma_list_to_gamma_argmax_list(gamma_list)
    gamma_argmax_list,gamma_argmax_list.shape    
       
    
 



    """


    with open('datasets/extract/{}.data.col'.format(dataset_name),'r') as f: exp_data_col=[i.strip().strip('"') for i in f.read().split()]
    with open('datasets/extract/{}.data.row'.format(dataset_name),'r') as f: exp_data_row=[i.strip().strip('"') for i in f.read().split()]
    allscore=allscore_munge


    crit1=allscore['QQdiff_rank'].copy()

    #(allscore[['PCvariance_rank','votingscore_rank']].min(axis=1)<(97/100*len(exp_data_row)))
    crit2=allscore['votingscore_rank'].copy()

    #(allscore[['PCvariance_rank','votingscore_rank']].min(axis=1)<(97/100*len(exp_data_row)))

    crit3=allscore['QQratio_rank'].copy()


    crit4=allscore['mean_0_all_rank'].copy()

    
    crit5=allscore['PCstd2_rank'].copy()    
    
    
    #(allscore[['votingscore_rank','PCvariance_rank']].min(axis=1)<(97/100*len(exp_data_row)))
    MarcoPolo_score=pd.concat([crit1,crit2,crit3,crit4,crit5],axis=1).sum(axis=1)    
    
    MarcoPolo_score[~(
            (allscore['minorsize']>int(10))&
            (allscore['minorsize']<int(70/100*len(exp_data_col)))        
            )]=len(allscore)  
    


    allscore['MarcoPolo']=MarcoPolo_score
    allscore['MarcoPolo_rank']=pd.Series(np.arange(allscore.shape[0]),index=allscore['MarcoPolo'].sort_values(ascending=True).index).loc[allscore.index]

    allscore_munge=allscore 


    allscore_munge['Symbol']=np.nan
    allscore_munge['Gene ID']=exp_data_row

    allscore_munge.to_csv('{}.MarcoPolo.{}.rank.munge.tsv'.format(path,mode),sep='\t')
    
    





    """





    
    marker_matrix=pd.DataFrame(index=exp_data_row, columns=exp_data_meta['phenoid'].unique())

    marker_matrix.head()   
    
    phenoid_count=np.array([np.sum((exp_data_meta['phenoid']==phenoid)) for phenoid in marker_matrix.columns])

    for idx,row in marker_matrix.iterrows():
        #print(exp_data_row.index(idx))
        exp_data_select=exp_data[exp_data_row.index(idx),:]
        gamma_argmax_select=gamma_argmax_list[exp_data_row.index(idx),:]
        phenoid_on=np.array([np.sum(gamma_argmax_select[(exp_data_meta['phenoid']==phenoid).values]==0) for phenoid in row.index])

        marker_matrix.loc[idx]=phenoid_on/phenoid_count    
        
    marker_matrix.to_csv('datasets/extract/{}.typefinding.MarcoPolo.tsv'.format(dataset_name),sep='\t')

Kohinbulk_filtered


/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning

HumanLiver_filtered
Zhengmix8eq_filtered
TabulaAorta_filtered
TabulaBladder_filtered
TabulaBrainMyeloid_filtered
TabulaBrainNonMyeloid_filtered
TabulaDiaphragm_filtered


/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


TabulaFat_filtered


...

/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning

Simul_10000_5e-4_9_filtered


/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning

Simul_10000_5e-4_10_filtered


/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning

In [21]:
allscore_munge.set_index('Gene name').loc['MALAT1']#ENSG00000168542

QQratio               2.82507
QQdiff                 3739.7
votingscore               120
PCvariance2           100.442
PCvariance3           95.1307
PCvariance5           81.6468
PCvariance10          50.4296
PCstd2                9.90716
PCstd3                9.66907
PCstd5                8.90343
PCstd10               6.61877
lfc                   0.61786
mean_0_all            61.3892
minorsize                3349
maxdiff              0.536974
ismarker                False
QQratio_rank              937
QQdiff_rank              1728
votingscore_rank          585
PCvariance2_rank          766
PCvariance3_rank          809
PCvariance5_rank          859
PCvariance10_rank         836
PCstd2_rank               781
PCstd3_rank               839
PCstd5_rank               999
PCstd10_rank              964
lfc_rank                 5878
mean_0_all_rank            28
minorsize_rank            256
all_rank                 2898
all_rank_rank             551
MarcoPolo                 585
MarcoPolo_

In [25]:
allscore_munge.iloc[:,:20]

,QQratio,QQdiff,votingscore,PCvariance2,PCvariance3,PCvariance5,PCvariance10,PCstd2,PCstd3,PCstd5,PCstd10,lfc,mean_0_all,minorsize,maxdiff,ismarker,QQratio_rank,QQdiff_rank,votingscore_rank,PCvariance2_rank
0,2.213668,1375.865335,10,196.210328,156.371087,107.680266,62.264988,13.053295,11.621340,9.319340,6.686472,0.290412,66.987486,3123,0.064384,False,1167,5967,1593,1741
1,2.246336,1394.072310,10,172.294480,138.765680,94.393434,56.128614,12.471771,11.137201,8.792256,6.461702,0.341455,57.774810,3078,0.175581,False,1147,5965,1614,1428
2,2.246303,1652.353049,10,190.511025,150.185441,102.258983,59.149801,12.869725,11.359390,9.018440,6.479894,0.333241,60.100834,2589,0.078044,False,1148,5941,1619,1666
3,2.130835,1372.395381,11,186.137301,147.922798,101.478413,59.817611,12.801510,11.352806,9.067719,6.629268,0.292788,36.613193,3048,0.074717,False,1196,5969,1573,1595
4,2.044709,1501.601831,10,182.477717,145.243756,99.817198,59.302744,12.615721,11.214760,8.979772,6.616982,0.261809,22.245148,3406,0.068590,False,1224,5951,1597,1550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,-0.489550,2084.881849,87,148.963851,115.014826,79.648610,49.242001,11.616116,10.032133,8.082066,6.156323,3.937719,1.025937,498,0.402425,False,1469,5842,715,1189
5998,-0.487656,2060.493077,1,257.013690,208.204171,156.775784,90.549830,15.524941,13.855276,11.843366,8.359902,3.933499,1.016015,486,0.494996,False,1465,5893,999999,4286
5999,-0.494291,2105.293758,1,271.664456,213.622962,158.600829,92.705567,15.966804,13.936614,11.838072,8.460400,3.624115,0.996538,525,0.154942,False,1487,5782,999999,5597
6000,-0.496559,2060.458317,1,257.232306,213.017751,163.366685,95.745814,15.874945,14.303940,12.268272,8.748076,3.630692,1.011748,487,0.194813,False,1498,5894,999999,4308


In [81]:
pd.concat([crit1,crit2,crit4,crit5],axis=1).

SyntaxError: invalid syntax (<ipython-input-81-c090575cb8d3>, line 1)

In [ ]:
ENSG00000280623

In [16]:
allscore_munge.set_index('Gene ID').loc['ENSG00000168542']#ENSG00000168542

QQratio                      9.05909
QQdiff                       843.967
votingscore                      105
PCvariance2                  80.1551
PCvariance3                  57.6148
PCvariance5                  61.3395
PCvariance10                 71.7395
PCstd2                        7.8203
PCstd3                       6.39366
PCstd5                       7.10207
PCstd10                      7.91963
lfc                          2.04414
mean_0_all                   2080.53
minorsize                         68
maxdiff                      1.54896
ismarker                        True
QQratio_rank                       1
QQdiff_rank                     1636
votingscore_rank                3911
PCvariance2_rank                  84
PCvariance3_rank                  62
PCvariance5_rank                  58
PCvariance10_rank                113
PCstd2_rank                       71
PCstd3_rank                       55
PCstd5_rank                       55
PCstd10_rank                     143
l

In [15]:
allscore_munge.set_index('Gene ID').loc['ENSG00000280623']

QQratio                      5.15273
QQdiff                       651.921
votingscore                      195
PCvariance2                  181.917
PCvariance3                  142.126
PCvariance5                  112.046
PCvariance10                 73.0044
PCstd2                         12.01
PCstd3                       10.6428
PCstd5                       9.64747
PCstd10                      7.65124
lfc                          1.59778
mean_0_all                   2935.79
minorsize                         28
maxdiff                      1.17299
ismarker                        True
QQratio_rank                     108
QQdiff_rank                     3563
votingscore_rank                1770
PCvariance2_rank                 334
PCvariance3_rank                 222
PCvariance5_rank                 166
PCvariance10_rank                120
PCstd2_rank                      232
PCstd3_rank                      182
PCstd5_rank                      143
PCstd10_rank                     114
l

In [ ]:
ENSG00000019485

In [ ]:
allscore_munge.sort_values('PCstd2_rank').iloc[:50]

In [82]:
allscore_munge.sort_values('MarcoPolo_rank').iloc[:50]

,QQratio,QQdiff,votingscore,PCvariance2,PCvariance3,PCvariance5,PCvariance10,PCstd2,PCstd3,PCstd5,PCstd10,lfc,mean_0_all,minorsize,maxdiff,ismarker,QQratio_rank,QQdiff_rank,votingscore_rank,PCvariance2_rank,PCvariance3_rank,PCvariance5_rank,PCvariance10_rank,PCstd2_rank,PCstd3_rank,PCstd5_rank,PCstd10_rank,lfc_rank,mean_0_all_rank,minorsize_rank,all_rank,all_rank_rank,MarcoPolo,MarcoPolo_rank,Gene name,Symbol,Gene ID
8402,5.409208,902.624219,833,48.325672,36.012907,31.827234,21.559676,6.067163,5.169615,5.104584,4.131487,2.039867,1214.388958,49,1.166551,True,67,1224,315,56,46,39,33,52,44,37,33,207,130,11736,1854,77,1721,0,ENSG00000147869,NaN,ENSG00000147869
6100,5.234859,825.494098,795,31.579696,24.950980,28.465651,24.669803,5.602084,4.874584,5.196045,4.685104,1.894093,3217.843115,40,1.200431,True,94,1798,332,48,38,36,34,51,41,39,36,543,18,12791,2462,120,2199,1,ENSG00000104371,NaN,ENSG00000104371
8374,5.321054,738.625308,681,112.551273,145.422053,135.999069,88.204950,8.782905,10.699093,10.830179,8.341826,1.843697,619.793136,22,0.797546,False,76,2590,392,145,235,271,220,99,190,242,201,750,430,14108,3374,225,3511,2,ENSG00000253507,NaN,ENSG00000253507
8047,7.195571,750.057073,718,70.309951,53.158261,46.284560,32.211296,7.101113,6.181482,6.083888,5.067259,1.938743,483.108069,75,1.021799,False,11,2471,362,72,56,45,41,62,51,42,39,407,673,8644,3195,201,3568,3,ENSG00000156574,NaN,ENSG00000156574
11196,7.407155,1063.216234,161,30.274822,22.271218,31.171886,27.665456,4.958790,4.140126,5.127653,4.968820,2.212786,431.758755,44,1.969475,True,8,436,2282,45,37,38,36,42,37,38,38,65,840,12290,5000,458,3600,4,ENSG00000136698,NaN,ENSG00000136698
12103,4.373563,895.029368,283,102.910228,91.899939,72.283377,50.270230,9.935691,9.410256,8.236184,6.695466,2.145335,366.606538,39,1.087388,True,400,1270,1074,129,103,76,57,138,117,87,70,91,1125,12877,3418,231,3607,5,ENSG00000174600,NaN,ENSG00000174600
13589,3.236507,893.197157,622,118.419235,151.737072,140.562090,90.291912,9.927995,11.544477,11.131820,8.578878,1.845592,273.202079,21,1.059182,True,3510,1283,431,152,258,305,238,137,250,281,244,741,1969,14140,2145,96,3820,6,ENSG00000188848,NaN,ENSG00000188848
12883,5.933228,1199.214968,163,49.163537,34.846338,40.790750,36.114157,6.217574,4.975842,5.763421,5.625132,2.222851,307.319095,34,4.154264,True,30,138,2240,57,43,40,45,55,43,41,48,63,1619,13285,4618,393,4052,7,ENSG00000016082,NaN,ENSG00000016082
10519,3.016825,712.090134,421,21.909120,16.929095,20.315018,17.391450,4.644421,3.976245,4.395028,4.063893,1.682781,589.863866,31,1.783328,True,5156,2883,677,36,34,34,32,39,36,36,32,1693,470,13561,4237,335,4069,8,ENSG00000006128,NaN,ENSG00000006128
13891,4.232909,950.589481,536,241.182363,212.740406,150.885573,86.485232,14.511790,13.835943,11.300874,7.883478,2.066973,250.669604,26,1.407617,True,506,934,523,750,734,376,207,548,628,303,147,161,2295,13930,1980,85,4300,9,ENSG00000106078,NaN,ENSG00000106078


In [ ]:
allscore_munge.iloc[exp_data_row.index('ENSG00000186272')]

In [ ]:
allscore_munge.iloc[exp_data_row.index('ENSG00000071282')]

In [ ]:
pd.set_option("display.max_columns", 100)

In [ ]:
allscore_munge.sort_values("MarcoPolo_rank")

In [ ]:
gamma_argmax_list

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

[                 Q            LL  em_idx_max  m_idx_max
 0     29047.506213 -29047.506213           1        100
 1     18979.895981 -18979.895981           1        100
 2     18204.063988 -18204.063988           1        100
 3     16145.744150 -16145.744150           2         80
 4     16002.743818 -16002.743818           1        100
 ...            ...           ...         ...        ...
 4709    387.201646   -387.201646           1       2340
 4710    387.228106   -387.228106           1       2180
 4711    381.486187   -381.486187           1       2300
 4712    433.158070   -433.158070           1       1980
 4713    390.000482   -390.000482           1       2280
 
 [4714 rows x 4 columns],
                  Q            LL  em_idx_max  m_idx_max
 0     16557.203286 -16557.203286           1        120
 1     12285.881091 -12285.881091           1        100
 2     12187.926401 -12187.926401           1        120
 3     11285.682047 -11285.682047           1        100
 4 

In [8]:
for dataset_name in dataset_name_all:
    print(dataset_name)
    path='datasets/extract/{}'.format(dataset_name)
    path
    
    exp_data=mmread('{}.data.counts.mm'.format(path)).toarray().astype(float)
    with open('{}.data.col'.format(path),'r') as f: exp_data_col=[i.strip().strip('"') for i in f.read().split()]
    with open('{}.data.row'.format(path),'r') as f: exp_data_row=[i.strip().strip('"') for i in f.read().split()]
    assert exp_data.shape==(len(exp_data_row),len(exp_data_col))
    assert len(set(exp_data_row))==len(exp_data_row)
    assert len(set(exp_data_col))==len(exp_data_col)        

    exp_data_meta=pd.read_csv('{}.metadatacol.tsv'.format(path),sep='\t')

    cell_size_factor=pd.read_csv('{}.size_factor.tsv'.format(path),sep='\t',header=None)[0].values.astype(float)#.reshape(-1,1)

    x_data_intercept=np.array([np.ones(exp_data.shape[1])]).transpose()
    x_data_null=np.concatenate([x_data_intercept],axis=1)

    #gamma_argmax_list=(exp_data<=1).astype(int)
    #(exp_data<=1).astype(int)
    gamma_argmax_list=(exp_data<=np.median(exp_data,axis=1).reshape(-1,1)).astype(int)

    #gamma_list=gamma_list_list[-1]    

    #gamma_argmax_list=QQ.gamma_list_exp_data_to_gamma_argmax_list(gamma_list,exp_data)#gamma_argmax_list=QQ.gamma_list_to_gamma_argmax_list(gamma_list)
    #gamma_argmax_list,gamma_argmax_list.shape    
       
    
    
    
    #allscore=pd.read_csv('{}.MarcoPolo.{}.rank.tsv'.format(path,mode),index_col=0,sep='\t')
    allscore=pd.read_csv('datasets/extract/{}.hvg.ht.pca.tsv'.format(dataset_name),sep='\t',index_col=0)
    allscore_munge=allscore.copy()
 
    
    marker_matrix=pd.DataFrame(index=exp_data_row, columns=exp_data_meta['phenoid'].unique())

    marker_matrix.head()   
    
    phenoid_count=np.array([np.sum((exp_data_meta['phenoid']==phenoid)) for phenoid in marker_matrix.columns])

    for idx,row in marker_matrix.iterrows():
        exp_data_select=exp_data[exp_data_row.index(idx),:]
        gamma_argmax_select=gamma_argmax_list[exp_data_row.index(idx),:]
        phenoid_on=np.array([np.sum(gamma_argmax_select[(exp_data_meta['phenoid']==phenoid).values]==0) for phenoid in row.index])

        marker_matrix.loc[idx]=phenoid_on/phenoid_count    
        
    marker_matrix.to_csv('datasets/extract/{}.typefinding.ht.pca.tsv'.format(dataset_name),sep='\t')
    

Kohinbulk_filtered
HumanLiver_filtered
Zhengmix8eq_filtered
TabulaAorta_filtered
TabulaBladder_filtered
TabulaBrainMyeloid_filtered
TabulaBrainNonMyeloid_filtered
TabulaDiaphragm_filtered
TabulaFat_filtered


/home/ch6845/tools/miniconda3/envs/MarcoPolo/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in true_divide


TabulaHeart_filtered
TabulaKidney_filtered
TabulaLargeIntestine_filtered
TabulaLimbMuscle_filtered
TabulaLiver_filtered
TabulaLung_filtered
TabulaMammaryGland_filtered
TabulaMarrow_filtered
TabulaPancreas_filtered
TabulaSkin_filtered
TabulaSpleen_filtered
TabulaThymus_filtered
TabulaTongue_filtered
TabulaTrachea_filtered


In [20]:
gamma_argmax_list.shape

(4714, 3994)

In [21]:
len(exp_data_row)

14694